# Data preparation - country attributes

In [ ]:
# imports
import pandas as pd
import pickle

In [ ]:
# load data
# meta data: country specific attributes
df_VarSpeccountry = pd.read_excel("country_data/country_attributes.xlsx", sheet_name=0)

# country specific attributes
df_country = pd.read_excel("country_data/country_attributes.xlsx", sheet_name=1)

# meta data: country pair specific data
df_VarSpeccountrypair = pd.read_excel("country_data/country_attributes.xlsx", sheet_name=2)

# country pair specific data
df_countrypair = pd.read_excel("country_data/country_attributes.xlsx", sheet_name=3)

# meta data: visa restrictions
df_VarSpecvisa = pd.read_excel("country_data/country_attributes.xlsx", sheet_name=4)

# country pair specific data
df_visa = pd.read_excel("country_data/country_attributes.xlsx", sheet_name=5)


In [ ]:
# Read in the Excel file 'country_attributes.xlsx' from the 'country_data' directory
# Load the second sheet in the file into a Pandas DataFrame, and set the first column as the index
df_country = pd.read_excel("country_data/country_attributes.xlsx", sheet_name=1, index_col=0)

# Remove rows from the DataFrame that have a value of 'YYY' in the 'ISO' column
df_country.drop(df_country.loc[df_country['ISO'] =='YYY'].index, inplace=True)

# Create a dictionary to map the freedom values to numerical values
d = dict()
d['NF'] = 0
d['PF'] = 0.5
d['F'] = 1
# Map the values in the 'freedom' column to the numerical values in the d dictionary
for i in range(2005, 2023):
    df_country['freedom' + str(i)] = df_country['freedom' + str(i)].map(d)

# Get the column names of the DataFrame which are categorical
categorical_columns = ['christian', 'muslim',
       'english_official', 'spanish_official', 'french_official',
       'english_used', 'spanish_used', 'french_used', 'OECD', 'schengen',
       'freedom2005', 'freedom2006', 'freedom2007', 'freedom2008',
       'freedom2009', 'freedom2010', 'freedom2011', 'freedom2012',
       'freedom2013', 'freedom2014', 'freedom2015', 'freedom2016',
       'freedom2017', 'freedom2018', 'freedom2019', 'freedom2020',
       'freedom2021', 'freedom2022', 'tropical', 'temperate', 'dry', 'polar',
       'continental']

# Impute values for the categorical columns in the DataFrame
for col in categorical_columns:
       df_country[col] = df_country[col].fillna(df_country[col].median())

# Get the column names of the DataFrame which are numerical
numerical_columns = df_country.columns.difference(categorical_columns).difference(['ISO'])

# Impute values for the numerical columns in the DataFrame
for col in numerical_columns:
       df_country[col] = df_country[col].fillna(df_country[col].mean())

# Initialize an empty DataFrame to store the country data for each year
df_country_per_year = pd.DataFrame()

# Iterate over the years from 2007 to 2021
for year in range(2007, 2022):
    # Make a copy of the relevant columns from the df_country DataFrame
    help_df = df_country[['ISO', 'christian', 'muslim','english_official', 'spanish_official', 'french_official',\
    'english_used', 'spanish_used', 'french_used', 'OECD', 'schengen', 'tropical', 'temperate', 'dry', 'polar','continental']].copy()
    # Add the GDPPC and logGDPPC columns for the current year
    help_df['GDPPC'] = df_country[f'GDPPC{year}']
    help_df['logGDPPC'] = df_country[f'logGDPPC{year}']
    # If the current year is 2017 or later, add the POP and logPOP columns
    if year>=2017:
        help_df['POP'] = df_country[f'POP{year}']
        help_df['logPOP'] = df_country[f'logPOP{year}']
    # Add the freedom column for the current year
    help_df['freedom'] = df_country[f'freedom{year}']

    # Add a "year" column to the help_df DataFrame with the current year repeated for each row
    help_df["year"] = [year]* len(help_df['ISO'])
    # Add the help_df DataFrame to the df_country_per_year DataFrame
    df_country_per_year = pd.concat([help_df, df_country_per_year])

# Rename the "ISO" and "year" columns in the df_country_per_year DataFrame
df_country_per_year = df_country_per_year.rename(columns = {"ISO": "COUNTRY_ISO3: Country ISO alpha-3 code", "year": "YEAR_WAVE: Wave Year"})
# Print the resulting df_country_per_year DataFrame
df_country_per_year

# save file as pickle
df_country_per_year.to_pickle("country_data/country_per_year.pickle")